In [1]:
pip install ipykernel

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
bancarrota=pd.read_csv("train_data.csv")
bancarrota_test=pd.read_csv("test_data.csv")
bancarrota.head()

,ID,Bankruptcy,Realized.Sales.Gross.Profit.Growth.Rate,Operating.Expense.Rate,Operating.Profit.Per.Share..Yuan...,Gross.Profit.to.Sales,Tax.rate..A.,Current.Liability.to.Liability,Continuous.Net.Profit.Growth.Rate,Long.term.Liability.to.Current.Assets,...,Per.Share.Net.profit.before.tax..Yuan...,Continuous.interest.rate..after.tax.,Liability.Assets.Flag,Working.Capital.to.Total.Assets,Degree.of.Financial.Leverage..DFL.,Operating.Gross.Margin,Contingent.liabilities.Net.worth,Operating.Profit.Growth.Rate,Cash.Flow.to.Liability,Quick.Assets.Total.Assets
0,1,0,0.087661,1.900349e-04,0.131094,0.583345,0.000000,0.815892,0.395719,0.089276,...,0.223720,0.801737,0,0.839112,0.036551,0.591350,0.007401,0.913175,0.489822,0.138714
1,2,0,0.021892,5.371576e-04,0.086167,0.695478,0.209649,0.768636,0.163505,0.000000,...,0.324763,0.791203,0,0.908450,0.126752,0.766066,0.020514,0.849241,0.550247,0.704211
2,3,0,0.102249,2.178329e-04,0.157755,0.766934,0.272061,0.643316,0.181571,0.010180,...,0.360892,0.803126,0,0.865943,0.018152,0.687211,0.004363,0.865814,0.447168,0.398086
3,4,0,0.029972,4.915756e+10,0.249279,0.611580,0.288357,0.752782,0.409196,0.018603,...,0.152445,0.824732,0,0.806254,0.051284,0.715973,0.014162,0.843357,0.485163,0.617386
4,6,0,0.021164,1.206001e-04,0.239817,0.614250,0.000000,0.559498,0.286989,0.148687,...,0.217282,0.787010,0,0.769843,0.012872,0.704043,0.003050,0.884905,0.422001,0.187121


PREPROCESAMIENTO

In [4]:
# Ver cuántos valores nulos hay por columna
bancarrota.isnull().sum().sort_values(ascending=False)


ID                                      0
Revenue.per.person                      0
Operating.Profit.Rate                   0
Long.term.fund.suitability.ratio..A.    0
Pre.tax.net.Interest.Rate               0
                                       ..
Cash.Flow.to.Sales                      0
Equity.to.Long.term.Liability           0
Working.Capital.Equity                  0
After.tax.net.Interest.Rate             0
Quick.Assets.Total.Assets               0
Length: 65, dtype: int64

In [6]:

# Variables más positivamente correlacionadas
vars_positivas = [
    'Debt.ratio..',                             # Razón de endeudamiento
    'Current.Liability.to.Current.Assets',      # Pasivo corriente / activo corriente
    'Current.Liability.to.Assets',              # Pasivo corriente / activos totales
    'Liability.Assets.Flag',                    # Indicador de pasivo / activos
    'Revenue.per.person'                        # Ingresos por persona
]
# Variables más negativamente correlacionadas
vars_negativas = [
    'Retained.Earnings.to.Total.Assets',        # Utilidades retenidas / activos totales
    'Working.Capital.to.Total.Assets',          # Capital de trabajo / activos totales
    'ROA.B..before.interest.and.depreciation.after.tax',  # ROA antes de intereses e impuestos
    'Tax.rate..A.',                             # Tasa impositiva
    'Quick.Assets.Total.Assets'                # Activo líquido / activos totales
]


In [10]:
from sklearn.preprocessing import StandardScaler

variables_seleccionadas = vars_positivas + vars_negativas

# Separar variables predictoras y objetivo en entrenamiento
X_train = bancarrota[variables_seleccionadas]
y_train = bancarrota['Bankruptcy']

# Estandarizar
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Preparar también el test set para predicciones de Kaggle
X_test = bancarrota_test[variables_seleccionadas]
X_test_scaled = scaler.transform(X_test)



PRIMER MODELO 

In [ ]:
#Multilayer Neural Network
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import AUC
import pandas as pd

# 1. Definir el modelo
modelo_nn = Sequential()
modelo_nn.add(Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)))
modelo_nn.add(Dropout(0.3))
modelo_nn.add(Dense(32, activation='relu'))
modelo_nn.add(Dropout(0.3))
modelo_nn.add(Dense(1, activation='sigmoid'))  # Salida para clasificación binaria

# 2. Compilar
modelo_nn.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[AUC(name='auc')]
)

# 3. Entrenar el modelo
early_stop = EarlyStopping(patience=5, restore_best_weights=True)

historial = modelo_nn.fit(
    X_train_scaled,
    y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

# 4. Predicciones sobre test
probabilidades = modelo_nn.predict(X_test_scaled).flatten()

# 5. Crear archivo de envío
submission = pd.DataFrame({
    'ID': bancarrota_test['ID'],
    'Bankruptcy': probabilidades
})

submission.to_csv('submission_red_neuronal.csv', index=False)
print("Archivo 'submission_red_neuronal.csv' creado.")


c:\Users\Yoselin\anaconda3\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - auc: 0.2340 - loss: 0.4762 - val_auc: 0.2413 - val_loss: 0.2348
Epoch 2/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - auc: 0.3051 - loss: 0.2474 - val_auc: 0.4697 - val_loss: 0.1670
Epoch 3/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - auc: 0.5627 - loss: 0.1579 - val_auc: 0.6382 - val_loss: 0.1492
Epoch 4/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - auc: 0.7888 - loss: 0.1182 - val_auc: 0.7339 - val_loss: 0.1402
Epoch 5/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - auc: 0.8035 - loss: 0.1189 - val_auc: 0.7599 - val_loss: 0.1375
Epoch 6/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - auc: 0.8019 - loss: 0.1037 - val_auc: 0.7873 - val_loss: 0.1341
Epoch 7/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - auc: 0.8901 - loss: 0.1000 - val_auc: 0.8037 - val_loss: 0.1331
Epoch 8/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - auc: 0.8567 - loss: 0.0884 - val_auc: 0.7890 - val_loss: 0.1336
Epoch 9/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - auc: 0.9094

SEGUNDO MODELO

In [14]:
#MLP Classifier
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X_train_scaled, y_train)

# Entrenar el modelo MLP con los datos balanceados
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(100,), 
                    activation='relu',
                    solver='adam',
                    max_iter=300,
                    random_state=42)

mlp.fit(X_resampled, y_resampled)


c:\Users\Yoselin\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(max_iter=300, random_state=42)

In [15]:
from sklearn.metrics import roc_auc_score, classification_report

# Predicción de probabilidades sobre el set de entrenamiento
y_train_pred_proba = mlp.predict_proba(X_train_scaled)[:, 1]

# Calcular AUC
auc_entrenamiento = roc_auc_score(y_train, y_train_pred_proba)
print(f"AUC en el set de entrenamiento: {auc_entrenamiento:.4f}")

# Clasificación básica (opcional, para revisar precisión, recall, etc.)
y_train_pred = mlp.predict(X_train_scaled)
print(classification_report(y_train, y_train_pred))


AUC en el set de entrenamiento: 0.9990
              precision    recall  f1-score   support

           0       1.00      0.97      0.99      1988
           1       0.55      1.00      0.71        62

    accuracy                           0.98      2050
   macro avg       0.77      0.99      0.85      2050
weighted avg       0.99      0.98      0.98      2050



In [16]:
# 1. Predecir probabilidades de clase 1
y_test_proba = mlp.predict_proba(X_test_scaled)[:, 1]

# 2. Obtener la columna ID desde el test original
IDs = bancarrota_test['ID']  # asegúrate de que se llame exactamente así

# 3. Crear el DataFrame de envío
envio = pd.DataFrame({
    'ID': IDs,
    'Bankruptcy': y_test_proba
})

# 4. Guardar como CSV
envio.to_csv('submission.csv', index=False)

print("✅ Archivo 'submission.csv' generado con éxito.")


✅ Archivo 'submission.csv' generado con éxito.
